<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Python Fundamentals: TITLE</h1>  

©2000-20 Chris Seddon 
</div>

Execute the following cell to activate styling for this tutorial

In [ ]:
from IPython.display import HTML
HTML(f"<style>{open('my.css').read()}</style>")

## Preamble: 1
In this tutorial, we will be showing how to wrap up C and C++ code into a module that can be called as if it were normal Python code; this is achieved by compiling C and C++ code into a shared library.

However, before we start this tutorial, I need to point out that Jupyter notebook doesn't always show the output of Python code.  This is because the notebook server runs code in a subshell and redirects this output of the shell to the browser.  But, some Python code runs in a subshell of a subshell and this output does not get redirected to the browser by the notebook server.

This is particularly true of Python "subprocess" module.  For example is we run code in the following cell, all we see is the reurn code from "subprocess.run()":

In [ ]:
import subprocess
subprocess.run("ls -l".split())

## Preamble: 2
We can circumvent this problem by using pipes.  The "subprocess" module can use pipes to get output from the subshell of the subshell as follows:

In [ ]:
import subprocess
result = subprocess.run("ls -l".split(), stdout=subprocess.PIPE)
print (f"return code: {result.returncode} \nstdout = \n{result.stdout.decode()}")

## Preamble: 3
For the rest of this tutorial, we will wrap up this functionality in a "run" function.  Sometimes, we will also need to specify the directory in which to execute the code:

In [ ]:
import subprocess
def run(cmd, dir="."):
    result = subprocess.run (cmd.split(), stdout=subprocess.PIPE, cwd=dir)
    print (f"return code = {result.returncode} \n{result.stdout.decode()}")
print("run command defined")

## Preamble: 4
Now we are ready to start the tutorial.  Since we are going to explore several examples, we will need several pre-written files.  I've decided to store these files under a "resources" folder for each example.  

We will "cd" to this folder now:

In [ ]:
%cd resources

## Preamble: 5
Now we are ready to start the tutorial.  Since we are going to explore several examples, we will need several pre-written files.  I've decided to store these files under a "resources" folder for each example.  The resources will be split according which example we are using:

In [ ]:
run("ls -l", "example1")
run("ls -l", "example2")

## Example1: 1
When we want to wrap up C and C++ code in a shared library we need to provide:
* the C/C++ code to go in the shared library
* setup.py: instructions on building the library

As an example of C code, we will write a Fibonacci module.  Here is the C code:

In [ ]:
run("cat fibmodule.c", "example1")

## Example1: 2
The C code is in 4 parts:

* the function<pre>int _fib(int n)</pre>
* binding code between C and Python in the function<pre>static PyObject* fib(PyObject* self, PyObject* args)</pre>
* interface definitions<pre>
method table
module definition structure</pre>
* Python entry point to the shared libary<pre>PyMODINIT_FUNC PyInit_fibonacci(void)</pre>  

The `setup.py` file is shown below:

In [ ]:
run("cat setup.py", "example1")

## Example1: 3
The key parts of the above file are:

* creating a "distutils.core.setup" object, containing build details <pre>setup(...)</pre>

* an Extension object defining which C files are to be compiled:
<pre>mymodule = Extension('fibonacci', sources = ['fibmodule.c'])</pre>

* the link between the "setup" object and the "Extension" object
<pre>ext_modules = [mymodule]</pre>

Python has a special way of building the shared library:

In [ ]:
import subprocess,os,sys

pythonPath = os.path.dirname(sys.executable)
swigPath = "/usr/local/bin"
compilerPath = "/opt/local/bin"
#os.environ["PATH"] = pythonPath + os.pathsep + os.environ["PATH"]
os.environ["PATH"] = pythonPath + os.pathsep + swigPath + os.pathsep + compilerPath + os.pathsep + os.environ["PATH"]

run("python setup.py -v build_ext", "example1")
run("ls -lR build", "example1")

## Example1: 4
Python creates a shared object file with a ".so" extension in the build directory a some other temporary files.

We now need to install this shared object .  Since we do not have permission to install in Diamond's Anacoda distribution we will have to install locally in our home directory:

In [ ]:
run("python setup.py install --record files.txt --user", "example1")

## Example1: 5
Now we've installed our library, we should clean the build area:

In [ ]:
run("python setup.py clean --all", "example1")
print("staging area cleaned")

## Example1: 6
Time to test our module:

In [ ]:
import fibonacci

print((fibonacci.fib(10)))
print((fibonacci.fib(11)))
print((fibonacci.fib(12)))

## Example1: 7
So our module is working!

Note, we've been keeping a record of the files we've been creating to facilitate the "uninstall":

In [ ]:
run("cat files.txt", "example1")

## Example1: 8
The following code will print out the "egg-info" file:

In [ ]:
installedFiles = open("example1/files.txt", "r")
for file in installedFiles:
    file = file.rstrip()
    if file.endswith("egg-info"):
        f = open(file, 'r')
        print(f.read())

## Example1: 9
Now we can unistall our module:

In [ ]:
try:
    installedFiles = open("example1/files.txt", "r")
    for file in installedFiles:
        print(f"rm {file}")
        run(f"rm {file}")

    print("rm example1/files.txt")
    run("rm example1/files.txt")
    print("Example uninstalled")
except:
    print("Example already uninstalled")

## Example2: 1
The above procedure in the same for C++.

The one difficulty with the above is writing the C code `fibmodule.c`.  If you look at the code again, realise that you would expect to have to write the equivalent of the function<pre>int _fib(int n)</pre>
but the binding code requires knowledge of internals of the Python interpretert.  It would be great if we didn't need to write the binding code; that is what the "Swig" library is for.  It also generates the interface tables and entry point.

For more complex examples the "Swig" library saves a lot of work.  We will look at how to modify the above procedure to use "Swig".

We'll be using a different example - a C file that has two messages.  Note that we are using "static" to avoid memory problems; we can't easily allocate memory for a char array in our C code and expect Python to clean it up:

In [ ]:
run("cat messages.c", "example2")

## Example2: 2
Swig needs a special file to tell it what binding code is required:

In [ ]:
run("cat messages.i", "example2")

## Example2: 3
The `setup.py` needs to be changed to reference the "swig" file ("messages.i"):

In [ ]:
run("cat setup.py", "example2")

## Example2: 4
Now we can run `setup.py` to generate binding code etc.  "swig" will be called automatically:

In [ ]:
run("python setup.py -v build_ext", "example2")

## Example2: 5
The binding code produced by "swig" is very verbose.  Here's the forst 100 lines ...

In [ ]:
run("head -100 messages_wrap.c", "example2")

## Example2: 6
Next, we can install:

In [ ]:
run("python setup.py install --record files.txt --user", "example2")

## Example2: 7
And test ...

In [ ]:
import mymessages

print(mymessages.say_hello("World"))
print(mymessages.say_goodbye("Universe"))

## Example 2: 8
Finally, clean up

In [ ]:
run("python setup.py clean --all", "example2")
run("rm messages_wrap.c mymessages.py", "example2")